In [506]:
from ipyleaflet import Map, Heatmap, FullScreenControl
from random import uniform
import numpy as np
import pandas as pd
import unicodedata
from unidecode import unidecode
import os

m = Map(center=(29, -111), zoom=7, scroll_wheel_zoom=True)
m.add_control(FullScreenControl())
m.layout.width='100%'
m.layout.height='800px'


In [507]:
def elimina_acentos(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

# Defunciones por COVID-19 en Sonora al día de hoy

In [508]:
#Archivo que contiene el nombre de los municipios de Sonora
file_nm = open('nombre_municipios_sonora.dat', 'r')
nm = [line.rstrip() for line in file_nm.readlines()]

#Las coordenadas se obtienen de datos del INEGI 2021
df_latlon = pd.read_csv('inegi_sonora_lat_lon.csv')
# print(df_latlon)

#Archivo que contiene el ID de los municipios de Sonora
catalogo = pd.read_excel('catalogos.xlsx',engine='openpyxl', sheet_name="Catálogo MUNICIPIOS")
catalogo.to_csv('catalogos.csv', encoding='utf-8')
df_catalogo = pd.read_csv('catalogos.csv', usecols=["CLAVE_MUNICIPIO","MUNICIPIO","CLAVE_ENTIDAD"])
df_catalogo["MUNICIPIO"] = df_catalogo["MUNICIPIO"].apply(unidecode)
catalogo_sonora = df_catalogo.loc[df_catalogo["CLAVE_ENTIDAD"]==26]
# print(catalogo_sonora)
    
#Archivo que contiene los casos COVID-19 en Sonora
df = pd.read_csv('municipios_sonora.csv', usecols=["MUNICIPIO_RES","FECHA_DEF"])
# print(df)

locations=[]
for mun in nm:
    salida = df_latlon.loc[df_latlon["Nom_Mun"]==mun]
    lat = salida.iloc[0][3]
    lon = salida.iloc[0][4]
    mun_may = np.char.upper(mun)    
    mun_norm = elimina_acentos(str(mun_may)) 
    id_mun = catalogo_sonora.loc[catalogo_sonora["MUNICIPIO"]==mun_norm]
    id_mun = id_mun.iloc[0][0]
    defunciones = len(df.loc[(df["MUNICIPIO_RES"]==id_mun) & (df["FECHA_DEF"] < "9999-99-99")])
#     print("Ciudad: ", mun_norm)
#     print("Latitud: ", lat)
#     print("Longitud: ", lon)
#     print("Defunciones: ", defunciones)
    locations.append([lat,lon,defunciones])

heatmap = Heatmap(locations=locations, radius=20)

m.add_layer(heatmap);
m

Map(center=[29, -111], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…